Pipeline
Pasos:
1. Preprocesa data (numerico y categorico), aquí se específica como se hace
2. Fit
3. Predict

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

X_full = pd.read_csv('C:\home\jupyter\data\d_train.csv', index_col='Id')
X_test_full = pd.read_csv('C:\home\jupyter\data\d_test.csv', index_col='Id')

#removing rows with missing target, separate target from predictors

X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace = True)

#  Separate

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [3]:
# lo mismo que en categorical_varibles, evitamos columnas de variables categoricas con más de 10 opciones 

#Select categorical columns with relatively low cardinality

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == 'object']

# select numerical columns 

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

## keep selected columns only

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()


In [9]:
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#Preprocessing for numerical data 
numerical_transformer = SimpleImputer(strategy='constant')

#Preprocessing for categorical data 
categorical_transformer = Pipeline( steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown = 'ignore'))])

# Bundle preprocessing for numerical and categorical data

preprocessor = ColumnTransformer(transformers = [('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])  # List of (name, transformer, column(s)) tuples specifying the transformer objects to be applied to subsets of the data.

#Define model 

model = RandomForestRegressor(n_estimators=100, random_state=0) 

#Bundle preprocessing and modeling code in a pipeline 
clf = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

#Preprocessing of training data, fit model
clf.fit(X_train, y_train)

#Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE', mean_absolute_error(y_valid, preds))

MAE 17861.780102739725


In [12]:
#Preprocessing for numerical data 
numerical_transformer = SimpleImputer(strategy = 'constant')

#Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')), ('onehot',OneHotEncoder(handle_unknown='ignore'))])

#Bundle preprocessing for numerical and categorical data 

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])

#Define the model

model = RandomForestRegressor(n_estimators=100, random_state=0)




In [13]:
#bundle preprocessing and modeling code in a pipeline 
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Preprocessing pf training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

#Evaluate the model
score = mean_absolute_error (y_valid, preds)

print('MAE', score)

MAE 17621.3197260274


In [15]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test)
preds_test

array([127168.41, 154869.75, 182907.65, ..., 148173.16, 109054.33,
       226853.  ])